In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [2]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install pdpipe 
!pip install symspellpy
!pip install pycontractions

     |████████████████████████████████| 40 kB 978 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.1-py3-none-any.whl size=30084 sha256=db5e6281f20c901a081f9415d98ffb509e285ed90d1573c99d6edab6bd6359f6
  Stored in directory: /root/.cache/pip/wheels/03/ae/77/ee98e1d17085ed67e01e617b7e5a24f320d3f00ad79846c939
  Created wheel for py-params: filename=py_params-0.9.6-py3-none-any.whl size=7088 sha256=c55690283318b2d5fb18230ed198178592183d187b6e8472042c91305fdb7ab5
  Stored in directory: /root/.cache/pip/wheels/d1/4d/5a/7fbbdfb87bf3da12265e0d79dc19ce143652a09e3696189eb8
  Created wheel for params-flow: filename=params_flow-0.8.0-py3-none-any.whl size=15999 sha256=4b5be7026f0052136e056dc57deaa915f9b03ef602384a93b70a90f1713c0a85
  Stored in directory: /root/.cache/pip/wheels/f7/ff/d3/cb47aae1024dbabf32cd5b983aa68042fa7bbcd173fcdc162c
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 47 kB 629 kB/s 
  Created wheel for decore: filenam

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam



########################################################
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub



try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert


#data pipeline 
import pdpipe as pdp

#Other Preprocessing
import re
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
import string
# !pip install symspellpy
import pkg_resources
from symspellpy.symspellpy import SymSpell
from symspellpy import SymSpell, Verbosity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd

import nltk
nltk.download('punkt')

#Contraction Import
from pycontractions import Contractions

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
train= pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [5]:
from bs4 import BeautifulSoup
import requests, json, csv
import re

def remove_wrong_abb(key,value):
    if(re.search('it means',value)):
        return False;
    elif(re.search('\*',value)):
        return False
    elif(value==""):
        return False
    elif(len(key)>7):
        return False
    else :
        return True
    
resp = requests.get("http://www.netlingo.com/acronyms.php")
soup = BeautifulSoup(resp.text, "html.parser")
slangdict= {}
key=""
value=""
for div in soup.findAll('div', attrs={'class':'list_box3'}):
    for li in div.findAll('li'):
        for a in li.findAll('a'):
            key =a.text
            value = li.text.split(key)[1]
            if(remove_wrong_abb(key,value)):
                if(re.search('\ -or- ',value)):
                    pos=re.search('\ -or- ',value).start()
                    slangdict[key]=value[:pos]
                else:
                    slangdict[key]=value

                    
# with open('myslang.json', 'w') as f:
#     json.dump(slangdict, f, indent=2)

w = csv.writer(open("myslang.csv", "w"))
for key, val in slangdict.items():
    w.writerow([key.lower(), val.lower()])
    
reader = csv.reader(open('myslang.csv', 'r'))
abbreviations = {}
for row in reader:
    k, v = row
    abbreviations[k] = v

In [6]:
sym_spell_4space = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell_4space.load_dictionary(dictionary_path, term_index=0, count_index=1)


sym_spell_misspelled = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
sym_spell_misspelled.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell_misspelled.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

True

In [7]:
cont = Contractions(api_key="glove-twitter-100")
cont.load_models()

[=====---------------------------------------------] 11.8% 45.8/387.1MB downloaded

In [8]:
#Version Edit: Sonam D.
def to_lower(text):
    text = text.lower()
    return text

#Version Edit: Anit G.
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+|pic.twitter.com\S+')
    return url.sub(r' ',text)

#Version Edit: Anit G.
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text)

#Version Edit: Anit G.
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

#Version Edit: Sonam D.
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

#Version Edit: Sonam D.
def remove_numbers(text):
    text = re.sub(r'\d+', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_mentions(text):
    text = re.sub(r'@\w*', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_punctuations(text):
    text = re.sub(r'([^A-Za-z \t])|(\w+:\/\/\S+)', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_square_bracket(text):
    text = re.sub('\[.*?\]', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_angular_bracket(text):
    text = re.sub('\<.*?\>+', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_newline(text):
    text = re.sub('\n', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_words_with_numbers(text):
    text = re.sub('\w*\d\w*', ' ', text)
    return text
    
#Version Edit: Susanth D.
def hashtag_to_words(text):
    text = re.sub(r'##', '#', text)
    hash_pattern = re.compile(r"#\w*")
    hashtag_list = re.findall(r"#\w+",text)
    for hashtag in hashtag_list:
        hashtag = re.sub(r'#', '', hashtag)
        text = re.sub(hashtag, sym_spell_4space.word_segmentation(hashtag).corrected_string, text)
    text = re.sub(r'#', '', text)
#     text = re.sub(r'# ', '', text)
    return text

#Version Edit: Susanth D.
def remove_stopwords(text):
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        if token not in stopwords.words('english'):
            textop = textop + token + ' '
    return textop

#Version Edit: Sonam D.
def correct_misspelled_with_context(text):
    suggestions = sym_spell_misspelled.lookup_compound(text, max_edit_distance=2)
    text = str(suggestions[0])
    text = re.sub(r', \d', ' ', text)
#     text = remove_numbers(text)
    return text

#Version Edit: Sonam D.
def stemming_text(text):
    stemmer= PorterStemmer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + stemmer.stem(token) + ' '
    return textop

#Version Edit: Sonam D.
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + lemmatizer.lemmatize(token) + ' '
    return textop

#Version Edit: Anit G.
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

#Version Edit: Anit G.
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

#Version Edit: Sonam D.
# def handle_contractions(text):
#     text = re.sub(r"’", "'", text)
#     for word in text.split():
#         if word.lower() in contractions:
#             text = text.replace(word, contractions[word.lower()])
#     return(text)

#Version Edit: Saurabh M.
def removeRepeated(tweet):
    prev = ''
    tweet_new = ''
    for c in tweet:
        caps = False
        if c.isdigit():
            tweet_new += c
            continue
        if c.isalpha() == True:
            if ord(c) >= 65 and ord(c)<=90:
                caps = True
            c = c.lower()
            if c == prev:
                count += 1
            else:
                count = 1
                prev = c
            if count >= 3:
                continue
            if caps == True:
                tweet_new += c.upper()
            else:
                tweet_new += c
        else:
            tweet_new += c
    return tweet_new


def Expand_Contractions(text):
    return list(cont.expand_texts([text]))[0]

In [9]:
def count_chars(text):
    new_text = text.apply(lambda x : list(x)).explode()
    return new_text.unique().shape[0]

def count_words(text):
    new_text = text.apply(lambda x : x.split(' ')).explode()
    return new_text.unique().shape[0]

def preprocess_pipeline(steps, col, df):
    new_col = df[col]
    char_count_before = 0
    word_count_before = 0
    char_count_after = 0
    word_count_after = 0
    for each_step in steps:
        char_count_before = count_chars(new_col)
        word_count_before = count_words(new_col)
        new_col = new_col.apply(each_step)
        char_count_after = count_chars(new_col)
        word_count_after = count_words(new_col)
        print("Preprocessing step: ",each_step.__name__)
        print("Char Count ---> Before: %d | After: %d"%(char_count_before, char_count_after))
        print("Word Count ---> Before: %d | After: %d"%(word_count_before, word_count_after))
    
    return new_col

In [10]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')


pipeline = []
# pipeline.append(to_lower)
pipeline.append(remove_newline)
pipeline.append(remove_URL)
pipeline.append(remove_html)
pipeline.append(remove_emoji)
# pipeline.append(hashtag_to_words)
# pipeline.append(remove_words_with_numbers)
# pipeline.append(remove_numbers)
pipeline.append(remove_mentions)
pipeline.append(remove_square_bracket)
pipeline.append(remove_angular_bracket)
pipeline.append(Expand_Contractions)
# pipeline.append(remove_punctuations)
pipeline.append(removeRepeated)
# pipeline.append(convert_abbrev_in_text)
# pipeline.append(remove_stopwords)
# pipeline.append(correct_misspelled_with_context)
# pipeline.append(remove_numbers)
# pipeline.append(remove_stopwords)
# # pipeline.append(stemming_text)
pipeline.append(lemmatization)

train['processed_text'] = preprocess_pipeline(pipeline, 'text', train)
test['processed_text'] = preprocess_pipeline(pipeline, 'text', test)

Preprocessing step:  remove_newline
Char Count ---> Before: 122 | After: 121
Word Count ---> Before: 32017 | After: 31925
Preprocessing step:  remove_URL
Char Count ---> Before: 121 | After: 121
Word Count ---> Before: 31925 | After: 27333
Preprocessing step:  remove_html
Char Count ---> Before: 121 | After: 121
Word Count ---> Before: 27333 | After: 27333
Preprocessing step:  remove_emoji
Char Count ---> Before: 121 | After: 121
Word Count ---> Before: 27333 | After: 27333
Preprocessing step:  remove_mentions
Char Count ---> Before: 121 | After: 120
Word Count ---> Before: 27333 | After: 25007
Preprocessing step:  remove_square_bracket
Char Count ---> Before: 120 | After: 120
Word Count ---> Before: 25007 | After: 24872
Preprocessing step:  remove_angular_bracket
Char Count ---> Before: 120 | After: 120
Word Count ---> Before: 24872 | After: 24872
Preprocessing step:  Expand_Contractions
Char Count ---> Before: 120 | After: 120
Word Count ---> Before: 24872 | After: 24700
Preprocessin

In [11]:
df = train.copy()
df_test = test.copy()

In [12]:
train['processed_text']

0       Our Deeds are the Reason of this # earthquake ...
1                Forest fire near La Ronge Sask . Canada 
2       All resident asked to 'shelter in place ' are ...
3       13,000 people receive # wildfire evacuation or...
4       Just got sent this photo from Ruby # Alaska a ...
                              ...                        
7608    Two giant crane holding a bridge collapse into...
7609    The out of control wild fire in California eve...
7610                   M1.94 ? 5km S of Volcano Hawaii . 
7611    Police investigating after an e-bike collided ...
7612    The Latest : More Homes Razed by Northern Cali...
Name: processed_text, Length: 7613, dtype: object

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


X_all = pd.concat([df["processed_text"], df_test["processed_text"]])
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf.fit(X_all)

X = tfidf.transform(df["processed_text"])
X_test = tfidf.transform(df_test["processed_text"])
del X_all

train, test = train_test_split(df, test_size=0.2)

train_x = train["processed_text"]
train_y = train["target"]

test_x = test["processed_text"]
test_y = test["target"]


X_train, X_val, y_train, y_val = train_test_split(X, df["target"], test_size=0.1, random_state=42)

In [14]:
parameters = { 
    'gamma': [0.001, 0.01, 0.1, 0.4, 0.5, 0.6, 0.7, 1], 
    'kernel': ['rbf'], 
    'C': [0.001, 0.01, 0.1, 1, 1.5, 2, 3, 10],
}
model = GridSearchCV(SVC(), parameters, cv=10, n_jobs=-1).fit(X_train, y_train)

In [15]:
model.cv_results_['params'][model.best_index_]

{'C': 1.5, 'gamma': 0.5, 'kernel': 'rbf'}

In [16]:
y_val_pred = model.predict(X_val)
accuracy_score(y_val, y_val_pred), f1_score(y_val, y_val_pred)

(0.7874015748031497, 0.7263513513513513)

In [17]:
confusion_matrix(y_val, y_val_pred)

array([[385,  41],
       [121, 215]])

In [18]:
y_test_pred = model.predict(X_test)

In [19]:
sub_df = pd.read_csv(os.path.join('../input/nlp-getting-started/', 'sample_submission.csv'))
sub_df["target"] = y_test_pred
sub_df.to_csv("submission.csv",index=False)